In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sys
from os.path import dirname

sys.path.append("/content/drive/MyDrive/FHDO/Sem2/Project")
sys.path.append("/content/drive/MyDrive/FHDO/Sem2/Project/dataset")

import shutil
src = '/content/drive/MyDrive/FHDO/Sem2/Project/dataset/road_following_B/'
dest = '/content/road_following_B/'
#destination = shutil.copytree(src, dest)
# Remove the destination directory if it already exists
shutil.rmtree(dest, ignore_errors=True)

# Copy the source directory to the destination
destination = shutil.copytree(src, dest)

src = '/content/drive/MyDrive/FHDO/Sem2/Project/xy_dataset.py'
dest = '/content/'
destination = shutil.copy2(src, dest)

src = '/content/drive/MyDrive/FHDO/Sem2/Project/utils.py'
dest = '/content/'
destination = shutil.copy2(src, dest)

# # Remove the destination directory if it already exists
# shutil.rmtree(dest, ignore_errors=True)

# # Copy the source directory to the destination
# destination = shutil.copytree(src, dest)

### Dataset Object

In [ ]:
import torchvision.transforms as transforms
from xy_dataset import XYDataset

TASK = 'road_following'

CATEGORIES = ['apex']

DATASETS = ['A', 'B']

TRANSFORMS = transforms.Compose([
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

datasets = {}
for name in DATASETS:
    datasets[name] = XYDataset(TASK + '_' + name, CATEGORIES, TRANSFORMS, random_hflip=True)


In [ ]:


import torchvision.transforms as transforms
from xy_dataset import XYDataset
import torchvision.models as models

TASK = 'road_following'

CATEGORIES = ['apex1_2']

DATASETS = ['A', 'B']

# Load the ResNet-34 model
model = models.resnet18(pretrained=True)
num_features = model.fc.in_features

# Modify the final fully connected layer to match the number of classes in your task
model.fc = torch.nn.Linear(num_features, len(CATEGORIES))

TRANSFORMS = transforms.Compose([
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
    transforms.Resize((224, 224)),  # Adjusted input size for ResNet-50
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

datasets = {}
for name in DATASETS:
    datasets[name] = XYDataset(TASK + '_' + name, CATEGORIES, TRANSFORMS, random_hflip=True)


In [ ]:
import ipywidgets
#choose set B
dataset = datasets[DATASETS[1]]

### Create model

In [ ]:
import torch
import torchvision
import torchvision.models as models

device = torch.device('cuda')
output_dim = 2 * len(dataset.categories)  # x, y coordinate for each category

# RESNET 34
#model = torchvision.models.resnet50(pretrained=True)
# model = torchvision.models.resnet34(pretrained=True)
# model.fc = torch.nn.Linear(512, output_dim)
# RESNET 18
#model = torchvision.models.resnet50(pretrained=True)
model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, output_dim)


model = model.to(device)
def load_model():
    model.load_state_dict(torch.load(), load_path)
def save_model():
    torch.save(model.state_dict(), save_path)


## Train

In [ ]:
BATCH_SIZE = 8
correct_range = 0.1

num_epoch = 20
#import numpy as np
#z = np.array([0,0,0,0,0,0,0,0])

optimizer = torch.optim.Adam(model.parameters())
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)

loss_widget = ipywidgets.FloatText(description='loss')
progress_widget = ipywidgets.FloatProgress(min=0.0, max=1.0, description='progress')

def train():
    global BATCH_SIZE, LEARNING_RATE, MOMENTUM, model, dataset, optimizer, eval_button, train_button, accuracy_widget, loss_widget, progress_widget, state_widget

    train_loader = torch.utils.data.DataLoader(
            dataset,
            batch_size=BATCH_SIZE,
            shuffle=True
        )

    model = model.train()

    accuracy = 0.0
    epoch = num_epoch
    while epoch > 0:
      print(f"iteration: {num_epoch-epoch}")
      i = 0
      sum_loss = 0.0
      error_count = 0.0
      for images, category_idx, xy in iter(train_loader):
        correct = 0.0
        # send data to device
        images = images.to(device)
        xy = xy.to(device)

        optimizer.zero_grad()

        # execute model to get outputs
        outputs = model(images)
        #print("outputs")
        #print(f" {0 + outputs[0][0]}")
        #print("xy")
        #print(xy)

        # compute MSE loss over x, y coordinates for associated categories
        loss = 0.0
        for batch_idx, cat_idx in enumerate(list(category_idx.flatten())):

          #print("batch_idx")
          #print(batch_idx)
          #print("cat_idx")
          #print(cat_idx)
          #print(" ")

          loss += torch.mean((outputs[batch_idx][2 * cat_idx:2 * cat_idx+2] - xy[batch_idx])**2)
        loss /= len(category_idx)

        #accuracy
        diffs = outputs - xy
        #print(diffs)
        for d in diffs:
          #print(d[0])
          if abs(d[0]) <= correct_range and  abs(d[1]) <= correct_range:
            #print(d)
            correct +=1.0

        accuracy = (accuracy + (correct/(BATCH_SIZE)))/2




        # run backpropogation to accumulate gradients
        loss.backward()

        # step optimizer to adjust parameters
        optimizer.step()

        # increment progress
        count = len(category_idx.flatten())
        i += count
        sum_loss += float(loss)
        progress_widget.value = i / len(dataset)
        loss_widget.value = sum_loss / i

      epoch = epoch - 1
      print(f"loss: {loss}")
      print(f"accuracy: {accuracy*100}")



In [ ]:
train()
#model = model.eval()

## SAVE THE TRAINED MODEL

In [ ]:

#save_path = "/content/model_resnet18_apex1.pth"
save_path = "/content/model_resnet18_20epoch_apex2_4_5.pth"
save_model()